<a href="https://colab.research.google.com/github/lsteffenel/M2Atmo_et_Climat/blob/main/exemple_age_detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimation de l'âge à partir d'une photo

Pour cet exemple, nous allons faire une classification avec des "tranches d'âge". Il est aussi possible de faire une régression, mais pour l'instant essayons la classification.

In [ ]:
import cv2
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Ici on télécharge une base d'images. Vous pouvez explorer les images en cliquant sur l'icone "dossier" à gauche

In [ ]:
!gdown 1nOR9p4nWpcWMtRCVTjEOGXzyM43LWcZX
!unzip -q combined_faces.zip

Dans cette étape, nous allons créer la partie X (variables) et la partie y (cible). On attribue les classes en fonction des tranches d'âge, ça serait presque similaire d'utiliser directement l'âge comme cible (dans le cas d'une régression, par exemple).

In [ ]:
all_images = os.listdir('combined_faces/')
#ranges = ['1-2','3-9','10-20','21-27','28-45','46-65','66-116']

X = []
y = []

l = len(all_images)

for a in range(l):
    X.append(cv2.imread(f'combined_faces/{all_images[a]}',0))
    age = int(all_images[a].split('_')[0])

    if age>=1 and age<=2:
        y.append(0)
    elif age>=3 and age<=9:
        y.append(1)
    elif age>=10 and age<=20:
        y.append(2)
    elif age>=21 and age<=27:
        y.append(3)
    elif age>=28 and age<=45:
        y.append(4)
    elif age>=46 and age<=65:
        y.append(5)
    elif age>=66 and age<=116:
        y.append(6)
    print(str(a)+'/'+str(l))

#np.savez_compressed('compressed image data.npz',x=X,y=y)

In [ ]:
#loaded = np.load('compressed image data.npz')

#X = loaded['x']
#y = loaded['y']

voici un exemple d'une image

In [ ]:
plt.imshow(X[1],cmap='gray')
print(y[1])

A partir du dataset, nous allons séparer les données en groupes d'entraînement et de test. Mais avant ça, indiquons que la cible est de type catégorique.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from keras.utils import to_categorical
y=to_categorical(y)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

Les images ont des formats diverses. Ici on oblige toutes à avoir la forme 200x200 pixels

In [ ]:
X_train = np.array(X_train).reshape(-1,200,200,1)

In [ ]:
X_test = np.array(X_test).reshape(-1,200,200,1)

In [ ]:
IMG_HEIGHT = 200
IMG_WIDTH = 200
IMG_SIZE = (IMG_HEIGHT,IMG_WIDTH)
batch_size = 128
epochs = 10

On utilise un Generator pour effectuer de l'augmentation de données (rotation, etc).

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   rotation_range=15)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data = train_datagen.flow(X_train,y_train,batch_size)

test_data = test_datagen.flow(X_test,y_test,batch_size)

Le modèle est composé de 4 couches "convolution+average pooling". Ensuite, on effectue encore un pooling avant les dernières couches denses utilisées pour la classification.

In [ ]:
final_cnn = Sequential()

final_cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(200, 200, 1)))    # 3rd dim = 1 for grayscale images.
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

final_cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

final_cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))

final_cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
final_cnn.add(AveragePooling2D(pool_size=(2,2)))


final_cnn.add(GlobalAveragePooling2D())

final_cnn.add(Dense(132, activation='relu'))

final_cnn.add(Dense(7, activation='softmax')) # 7 car on a 7 "classes d'âge"

final_cnn.summary()

In [ ]:
final_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Enfin, on rajoute une "astuce" : sauvegarder le meilleur modèle afin de l'utiliser après l'entraînement.

In [ ]:
checkpoint = ModelCheckpoint(filepath="models/final_cnn_model_checkpoint.keras",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

In [ ]:
history = final_cnn.fit(train_data,
                    batch_size=512,
                    validation_data=test_data,
                    epochs=10,
                    callbacks=[checkpoint],
                    shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                    )

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
plotting_data_dict = history.history

plt.figure(figsize=(12,8))

test_loss = plotting_data_dict['val_loss']
training_loss = plotting_data_dict['loss']
test_accuracy = plotting_data_dict['val_accuracy']
training_accuracy = plotting_data_dict['accuracy']

epochs = range(1,len(test_loss)+1)

plt.subplot(121)
plt.plot(epochs,test_loss,marker='X',label='test_loss')
plt.plot(epochs,training_loss,marker='X',label='training_loss')
plt.legend()

plt.subplot(122)
plt.plot(epochs,test_accuracy,marker='X',label='test_accuracy')
plt.plot(epochs,training_accuracy,marker='X',label='training_accuracy')
plt.legend()

#plt.savefig('training.png')

#On fait un test ?

Déposez une image. Essayez d'avoir une image cadrée sur votre visage, pour augmenter la réussite.

Dans le prochaine paragraphe, changez "nom_fichier.png" par celui de votre image.

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/final_cnn_model_checkpoint.keras")
ranges = ['1-2','3-9','10-20','21-27','28-45','46-65','66-116']

image = cv2.imread("nom_fichier.png")

face = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

face = cv2.resize(face,(200,200))
face = face.reshape(1,200,200,1)
age = model.predict(face)
agerange = ranges[np.argmax(age)]

plt.imshow(image, cmap='gray')
print ("votre âge est dans la tranche", agerange)